In [ ]:
import torch
import pandas as pd
import os  # Добавляем модуль os

In [ ]:
# Загрузка данных (если они еще не загружены)
df = pd.read_csv('data/RIA_before_2025.csv').dropna().reset_index(drop=True).reset_index()[['index', 'text']]
df.to_csv('to_gemini.csv', index=False)  # Эта строка уже была, но я добавил её для полноты

# Определение размера чанка
chunk_size = 3000

# Общее количество строк
total_rows = len(df)

# Вычисление количества файлов (с округлением вверх)
num_files = (total_rows + chunk_size - 1) // chunk_size

# Создание папки для сохранения файлов
output_folder = 'gemini_chunks'  # Имя папки
if not os.path.exists(output_folder):  # Проверяем, существует ли папка
    os.makedirs(output_folder)  # Создаем папку, если она не существует


for i in range(num_files):
    # Вычисление начального и конечного индекса для текущего чанка
    start_index = i * chunk_size
    end_index = min((i + 1) * chunk_size, total_rows)  # min нужен, чтобы не выйти за пределы

    # Создание DataFrame для текущего чанка
    chunk_df = df.iloc[start_index:end_index]

    # Сохранение в файл
    filename = f'to_gemini_part_{i+1}.csv'  # Нумерация с 1
    filepath = os.path.join(output_folder, filename) # Полный путь к файлу, включая папку
    chunk_df.to_csv(filepath, index=False)

    print(f"Создан файл: {filename} (строки с {start_index} по {end_index -1})")

print("Разбиение завершено.")

Создан файл: to_gemini_part_1.csv (строки с 0 по 2999)
Создан файл: to_gemini_part_2.csv (строки с 3000 по 5999)
Создан файл: to_gemini_part_3.csv (строки с 6000 по 8999)
Создан файл: to_gemini_part_4.csv (строки с 9000 по 11999)
Создан файл: to_gemini_part_5.csv (строки с 12000 по 14999)
Создан файл: to_gemini_part_6.csv (строки с 15000 по 17999)
Создан файл: to_gemini_part_7.csv (строки с 18000 по 20999)
Создан файл: to_gemini_part_8.csv (строки с 21000 по 23999)
Создан файл: to_gemini_part_9.csv (строки с 24000 по 26999)
Создан файл: to_gemini_part_10.csv (строки с 27000 по 29999)
Создан файл: to_gemini_part_11.csv (строки с 30000 по 32999)
Создан файл: to_gemini_part_12.csv (строки с 33000 по 35999)
Создан файл: to_gemini_part_13.csv (строки с 36000 по 38999)
Создан файл: to_gemini_part_14.csv (строки с 39000 по 41999)
Создан файл: to_gemini_part_15.csv (строки с 42000 по 44999)
Создан файл: to_gemini_part_16.csv (строки с 45000 по 47999)
Создан файл: to_gemini_part_17.csv (строки 

Ты — редактор новостного агрегатора в России. Твоя задача: категоризировать новости ('news') для российских читателей, создав две колонки: 'multi _labels и
*hier label'. Учитывай, что эти данные будут использоваться моделью Longformer для мультиклассовой классификации, поэтому *количество уникальных классов должно быть ограничено*.
1. ***multi labels ** Перечисли *наиболее значимые* категории новости через точку с запятой (;), в порядке убывания вероятности. Используй
*ограниченный набор* категорий (не более 5-7). Выбирай наиболее общие и репрезентативные категории, которые охватывают основные темы новости.
Избегай слишком узких или специфичных категорий.
* **Примеры категорий:** Политика, Экономика, Общество, Происшествия, Спорт, Культура, Технологии, Международные отношения, Региональные новости. (Можно добавить/изменить список, но он должен быть конечным и не слишком большим).
* **Плохой пример:** "Политика; Экономика; Внутренняя политика; Законодательство; Налоги; Пенсионная реформа" (слишком много и слишком подробно)
* **Хороший пример:** "Политика; Экономика; Общество"
2. **'hier label :** Для *первой* категории из multi labels создай двухуровневую иерархию (уровень!/уровень2). Уровень2 должен *уточнять* Уровень! и быть
*более конкретным*, чем любая категория из 'multi _labels*, но *оставаться в рамках разумного количества уникальных значений*. Используй ограниченный набор уточнений для второго уровня.
* **Плохой пример:** "Политика/Встреча президента Иванова с премьер-министром Петровым 23 мая 2024 года в Кремле" (слишком специфично)
* **Хороший пример:** "Политика/Внутренняя политика"
* **Хороший пример:** "Происшествия/ДТП"
3. **Контекст:** Анализируй с точки зрения российского гражданина, объективно, но учитывая российские реалии.
Добавь только колонки 'multi labels' и 'hier label' к исходной news'.

In [17]:
import ast

labels_df = pd.read_json('result.json')

# # Преобразование 'hier_label' в списки (аналогично multi_labels)
# df['hier_label'] = df['hier_label'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
# # Преобразование 'multi_labels' в списки (на всякий случай, еще раз)
# df['multi_labels'] = df['multi_labels'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

labels_df

,index,multi_labels,hier_label
0,0,"[Международные отношения, Политика, Происшествия]","[Международные отношения, Конфликты]"
1,1,"[Международные отношения, Политика, Вооружение]","[Международные отношения, Политика]"
2,2,"[Общество, Региональные новости, Политика]","[Общество, Праздники]"
3,3,"[Международные отношения, Происшествия, Политика]","[Международные отношения, Протесты]"
4,4,"[Международные отношения, Религия, Общество]","[Международные отношения, Религия]"
...,...,...,...
219,219,"[Происшествия, Транспорт, Региональные новости]","[Происшествия, ЧП]"
220,220,"[Общество, Международные отношения, Свобода сл...","[Общество, Свобода слова]"
221,221,"[Международные отношения, Политика]","[Международные отношения, Политика]"
222,222,"[Международные отношения, Религия, Общество]","[Международные отношения, Религия]"


In [19]:
labels_df[labels_df['index']==93]

,index,multi_labels,hier_label
93,93,"[Международные отношения, Происшествия, Престу...","[Международные отношения, Преступность]"


In [14]:
df = pd.read_csv('data/RIA_before_2025.csv').dropna().reset_index(drop=True).reset_index()
df

,index,date,year,month,day,text
0,0,2020-01-01 00:32:53,2020,1,1,"Трамп вновь пригрозил последствиями Ирану, есл..."
1,1,2020-01-01 00:54:16,2020,1,1,Ким Чен Ын пообещал скоро представить новое ст...
2,2,2020-01-01 12:44:25,2020,1,1,"В России почти 6,5 миллиона человек встретили ..."
3,3,2020-01-01 12:55:30,2020,1,1,Столкновения произошли в Гонконге во время сог...
4,4,2020-01-01 15:20:14,2020,1,1,"Папа Римский Франциск извинился за то, что нак..."
...,...,...,...,...,...,...
168730,168730,2024-12-31 18:21:42,2024,12,31,Заявка на транзит российского газа через Украи...
168731,168731,2024-12-31 19:45:52,2024,12,31,\n\n▪️Российские войска нанесли групповой удар...
168732,168732,2024-12-31 21:02:19,2024,12,31,"❗️Путин заслушал доклады начальника Генштаба, ..."
168733,168733,2024-12-31 21:05:02,2024,12,31,Повторные выбросы нефтепродуктов зафиксированы...


In [15]:
merged_df = pd.merge(df, labels_df, on='index')
merged_df

,index,date,year,month,day,text,multi_labels,hier_label
0,0,2020-01-01 00:32:53,2020,1,1,"Трамп вновь пригрозил последствиями Ирану, есл...","[Международные отношения, Политика, Происшествия]","[Международные отношения, Конфликты]"
1,1,2020-01-01 00:54:16,2020,1,1,Ким Чен Ын пообещал скоро представить новое ст...,"[Международные отношения, Политика, Вооружение]","[Международные отношения, Политика]"
2,2,2020-01-01 12:44:25,2020,1,1,"В России почти 6,5 миллиона человек встретили ...","[Общество, Региональные новости, Политика]","[Общество, Праздники]"
3,3,2020-01-01 12:55:30,2020,1,1,Столкновения произошли в Гонконге во время сог...,"[Международные отношения, Происшествия, Политика]","[Международные отношения, Протесты]"
4,4,2020-01-01 15:20:14,2020,1,1,"Папа Римский Франциск извинился за то, что нак...","[Международные отношения, Религия, Общество]","[Международные отношения, Религия]"
...,...,...,...,...,...,...,...,...
219,219,2020-01-10 09:21:55,2020,1,10,"Вертолет Ми-8 ""Красавиа"" совершил экстренную п...","[Происшествия, Транспорт, Региональные новости]","[Происшествия, ЧП]"
220,220,2020-01-10 09:32:03,2020,1,10,Европейская федерация журналистов (EFJ) направ...,"[Общество, Международные отношения, Свобода сл...","[Общество, Свобода слова]"
221,221,2020-01-10 11:16:06,2020,1,10,Трамп передал советнику по безопасности презид...,"[Международные отношения, Политика]","[Международные отношения, Политика]"
222,222,2020-01-10 11:23:08,2020,1,10,В посольстве Ирана в Москве открыли книги собо...,"[Международные отношения, Религия, Общество]","[Международные отношения, Религия]"


In [20]:
merged_df[merged_df['index']==93]

,index,date,year,month,day,text,multi_labels,hier_label
93,93,2020-01-06 19:13:29,2020,1,6,Пираты напали на судно и похитили трех членов...,"[Международные отношения, Происшествия, Престу...","[Международные отношения, Преступность]"


In [16]:
merged_df.to_csv('merged_df.csv', index=False)